# **2 Step** - Curated (Transforme) **SILVER**

In [1]:
# Bibliotecas necessárias
import os
import pandas as pd
import sqlite3
import time
import logging

/home/jvcampos/projects/etl-voos-anac/.venv/lib/python3.11/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
# Carregamento da base de dados
df_ocorrencias = pd.read_csv('../data/raw/V_OCORRENCIA_AMPLA.csv', sep=';', encoding='utf-8', skiprows=1)

In [3]:
df_ocorrencias.head()

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,25043,202019110,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,Incidente,2020-06-17,22:10,ALTO ALEGRE,RR,Norte,OUTROS,...,NaN,NaN,NaN,182K,L1P,C182,1270.0,4.0,CESSNA AIRCRAFT,verdadeiro
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
2,25503,202023366,AZUL,Incidente,2020-10-02,19:02,CAMPINAS,SP,Sudeste,FOGO/FUMAÇA,...,NaN,NaN,NaN,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,verdadeiro
3,25623,202021754,JOAO MAURICIO GOTTARDI LOPES,Incidente,2020-10-07,15:30,ARAÇATUBA,SP,Sudeste,CONTATO ANORMAL COM A PISTA,...,NaN,NaN,NaN,180J,L1P,C180,1270.0,6.0,CESSNA AIRCRAFT,verdadeiro
4,25624,202004387,JOSE LUIZ CUNHA RODRIGUES,Incidente,2020-10-06,15:25,SÃO PAULO,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,NaN,NaN,NaN,M20J,L1P,M20P,1243.0,4.0,MOONEY AIRCRAFT,verdadeiro


In [4]:
df_ocorrencias.dtypes

Numero_da_Ocorrencia                  int64
Numero_da_Ficha                      object
Operador_Padronizado                 object
Classificacao_da_Ocorrencia          object
Data_da_Ocorrencia                   object
Hora_da_Ocorrencia                   object
Municipio                            object
UF                                   object
Regiao                               object
Descricao_do_Tipo                    object
ICAO                                 object
Latitude                             object
Longitude                            object
Tipo_de_Aerodromo                    object
Historico                            object
Matricula                            object
Categoria_da_Aeronave                object
Operador                             object
Tipo_de_Ocorrencia                   object
Fase_da_Operacao                     object
Operacao                             object
Danos_a_Aeronave                     object
Aerodromo_de_Destino            

In [5]:
sorted(df_ocorrencias["Municipio"].dropna().unique())

['\tCAMPO GRANDE',
 '\tCAMPO GRANDE ',
 '\tCAXIAS DO SUL',
 '\tFERNANDO DE NORONHA ',
 '\tITAITUBA',
 '\tPORTO ALEGRE',
 '\tPORTO ALEGRE ',
 '\tRIO DE JANEIRO',
 '\tRIO DE JANEIRO ',
 '\tSANTANA DO ARAGUAIA',
 '\tSÃO GONÇALO DO AMARANTE',
 '\tSÃO JOSÉ DOS PINHAIS',
 '\tSÃO PAULO',
 '\tSÃO PAULO ',
 '\tVÁRZEA GRANDE ',
 '  RIO DE JANEIRO ',
 ' BELO HORIZONTE',
 ' CATANDUVA ',
 ' Cidade do México, México',
 ' ESTADOS UNIDOS',
 ' FORTALEZA',
 ' Longitude Histórico',
 ' RIO DE JANEIRO',
 ' SÃO FÉLIX DO ARAGUAIA',
 ' SÃO JOSÉ DO RIO PRETO',
 ' SÃO PAULO ',
 ' VILA VELHA',
 '12:54',
 '22:48:36-S',
 '23:25:48-S',
 'ABADIA DE GOIÁS',
 'ACARÁ',
 'ACREÚNA',
 'AEROPORTO DE GUARULHOS',
 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS',
 'AFONSO CLÁUDIO',
 'AFUÁ',
 'AGUA BOA',
 'AGUASCALIENTES, MÉXICO',
 'AGUAÍ',
 'AGUDO',
 'AGUDOS',
 'ALCÂNTARA',
 'ALEGRETE',
 'ALEGRETE ',
 'ALFENAS',
 'ALFREDO CHAVES',
 'ALIANÇA DOS TOCANTINS',
 'ALMEIRIM',
 'ALMIRANTE TAMANDARÉ',
 'ALPESTRE ',
 'ALTA FLORESTA',
 "ALTA FLOREST

In [6]:
df_ocorrencias["Municipio"] = df_ocorrencias["Municipio"].str.strip()
sorted(df_ocorrencias["Municipio"].dropna().unique())

['12:54',
 '22:48:36-S',
 '23:25:48-S',
 'ABADIA DE GOIÁS',
 'ACARÁ',
 'ACREÚNA',
 'AEROPORTO DE GUARULHOS',
 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS',
 'AFONSO CLÁUDIO',
 'AFUÁ',
 'AGUA BOA',
 'AGUASCALIENTES, MÉXICO',
 'AGUAÍ',
 'AGUDO',
 'AGUDOS',
 'ALCÂNTARA',
 'ALEGRETE',
 'ALFENAS',
 'ALFREDO CHAVES',
 'ALIANÇA DOS TOCANTINS',
 'ALMEIRIM',
 'ALMIRANTE TAMANDARÉ',
 'ALPESTRE',
 'ALTA FLORESTA',
 "ALTA FLORESTA D'OESTE",
 'ALTAIR',
 'ALTAMIRA',
 'ALTO ALEGRE',
 'ALTO ALEGRE DO PINDARÉ',
 'ALTO ARAGUAIA',
 'ALTO GARÇAS',
 'ALTO HORIZONTE',
 'ALTO PARAGUAI',
 'ALTO PARANÁ',
 'ALTO PARAÍSO',
 'ALTO PARAÍSO DE GOIÁS',
 'ALTO PIQUIRI',
 'ALTO SANTO',
 'ALTÔNIA',
 'AMAJARI',
 'AMAPORÃ',
 'AMAPÁ',
 'AMERICANA',
 'AMERICANO DO BRASIL',
 'ANAJÁS',
 'ANAMÃ',
 'ANANINDEUA',
 'ANAPU',
 'ANAPÓLIS',
 'ANAURILÂNDIA',
 'ANDRADINA',
 'ANGRA DOS REIS',
 'ANÁPOLIS',
 'APARECIDA DE GOIÂNIA',
 'APUCARANA',
 'APUÍ',
 'AQUIDAUANA',
 'AQUIRAZ',
 'ARACAJU',
 'ARACAJÚ',
 'ARACATI',
 'ARACRUZ',
 'ARAGARÇAS',
 'ARA

In [7]:
df_ocorrencias["Municipio"] = df_ocorrencias["Municipio"].str.strip()
num_distintos = df_ocorrencias["Municipio"].dropna().nunique()
print(f"Número de municípios distintos: {num_distintos}")


Número de municípios distintos: 1439


In [8]:
df_ocorrencias["Municipio"].isnull().sum()  # Conta só os nulos da coluna "Municipio"

np.int64(3698)

In [9]:
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

#O que fazer com esses dados?
df_ocorrencias[
    df_ocorrencias["Municipio"].isnull() &
    df_ocorrencias["Regiao"].isnull() &
    df_ocorrencias["UF"].isnull() &
    df_ocorrencias["Latitude"].isnull() &
    df_ocorrencias["Longitude"].isnull()
]

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
7963,25805,202067408,AMERICAN AIRLINES,Acidente,2020-10-24,15:53,NaN,NaN,NaN,EXCURSÃO DE PISTA,MYGF,NaN,NaN,NaN,Aircraft landed on RWY 24 and shortly after making contact with the runway veered off of the runway resulting in a runway excursion. There were no injuries reported.,N674RJ,Estrangeira,AMERICAN AIRLINES,RE,Corrida após pouso,Voo Regular,NaN,MYGF,KMIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
11271,40149,202318593475,TAM,Incidente,2023-10-29,17:45:00,NaN,NaN,NaN,COLISÃO COM AVE,Fora de Aeródromo,NaN,NaN,-,Houve colisão com ave.,PRMHQ,TPR,TAM LINHAS AEREAS S.A,BIRD,Indeterminada,Voo Regular,Leve,SBSP,SBUL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A320-214,L2J,A320,77000.0,189.0,AIRBUS S.A.S.,verdadeiro
11457,36986,2023626632,CARGOLUX AIRLINES INTERNATIONAL S.A.,Incidente Grave,2018-01-10,03:10:00,NaN,NaN,NaN,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Fora de Aeródromo,NaN,NaN,-,"During cruise between WPT 0524N and KODOS, at time 03:10Z the crew experienced an engine malfunction. The first indication was a smell/fumes in the Flight Deck. The crew don the crew oxygen masks followed by a smoke alarm in the R/H crew rest bunk. In mean time smoke was present in the entire crew rest area. thereafter the crew received an engine vibration indication and the engine #1EGT increased to 900 degree Celsius. The crew performed the required non-normal checklist. the smoke immediately seized while observing the engine #1 oil quantily being at zero. The crew assumed the source of the smoke was the oil leak on the failed engine#1. With the engine secured and no more smoke present, the crew decided to divert to Recife for a safe landing. the crew declared a Pan Pan situation with ATC and landed safely. The aircraft is presently AOG.",LXUCV,Estrangeira,CARGOLUX AIRLINES INTERNATIONAL S.A.,SCF-PP,Cruzeiro,Voo Regular,NaN,SBKP,ELLX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso


In [10]:
df_municipios = pd.read_csv('../data/raw/municipios.csv', sep=';', encoding='latin1')
df_municipios.head()

,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
0,1,1100106,GUAJARÁ-MIRIM,Guajará-Mirim,RO
1,2,1100379,ALTO ALEGRE DOS PARECIS,Alto Alegre dos Parecis,RO
2,3,1100205,PORTO VELHO,Porto Velho,RO
3,4,1100452,BURITIS,Buritis,RO
4,5,1100122,JI-PARANÁ,Ji-Paraná,RO


In [11]:
# Converta para sets, eliminando nulos e garantindo que tudo seja string
todos_municipios = set(df_municipios['MUNICÍPIO - TOM'].dropna().astype(str).str.strip().str.upper())
municipios_incidentes = set(df_ocorrencias['Municipio'].dropna().astype(str).str.strip().str.upper())

# Encontre os valores exclusivos de cada conjunto
municipios_so_em_um = (todos_municipios ^ municipios_incidentes)

# Ordenar os que estão apenas em df_ocorrencias
municipios_somente_em_df_incidentes = sorted(municipios_incidentes - todos_municipios)

print(municipios_somente_em_df_incidentes)


['12:54', '22:48:36-S', '23:25:48-S', 'AEROPORTO DE GUARULHOS', 'AEROPORTO DE KIMBERLEY', 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS', 'AGUA BOA', 'AGUASCALIENTES, MÉXICO', 'ALIANÇA DOS TOCANTINS', 'ANAPÓLIS', 'ARACAJÚ', 'ARGENTINA', 'ARIPUAÑA', 'ASSIS CHATEAUBRIAN', 'BAIRRO MULTIRÃO I', 'BALNEARIO CAMBORIU', 'BARRA DO GARCAS', 'BASE AÉREA DE SANTOS', 'BELA VISTA DO PARAISO', 'BELO HORIZONE', 'BIRITIBA MIRIM', 'BLUMENAL', 'BOA VISTA DOS RAMOS', 'BOM JESUS DE GOIÁS', 'BONOPÓLIS', 'BRAISLÂNDIA', 'BRASILIA', 'BRASÌLIA', 'BRASÍLIA,', 'BRASÍLIADF', 'BUENOS AIRES, ARGENTINA', 'CAIXIAS DO SUL', 'CAMOINAS', 'CAMPINA GRANDE |', 'CAMPO DE MARTE', 'CAMPO DOS GOYTACAZES', 'CAMPO GRANDEMS', 'CAMPO GRNADE', 'CAMPO MORÃO', 'CAMPOS', 'CAMPOS DOS GOITACAZES', 'CAMPOS DOS GOYTACAS', 'CAMPOS DOS GOYTACASES', 'CAMPPINAS', 'CANCUN, MÉXICO', 'CANCÚN, MÉXICO', 'CAPE TOWN', 'CAPÃO DO CANOA', 'CARLÂNDIA', 'CASIMIRO E ABREU', 'CHAPADA DOS GUIMA', 'CHITOSE, JAPÃO', 'CIDADE DE UNA', 'CIDADE DO MÉXICO, MÉXICO', 'COLIDER', 

In [12]:
# Exibir todas as linhas onde a coluna 'Municipio' é igual a 'SÃO PAULO'
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

# REMOVER OU MANTES ESSAS LINHAS ?
# df_ocorrencias[df_ocorrencias['Municipio'] == 'AGUASCALIENTES, MÉXICO']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'ARGENTINA']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'BUENOS AIRES, ARGENTINA']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'CAPE TOWN']


# Não encontrei valores:
# df_ocorrencias[df_ocorrencias['Municipio'] == 'AEROPORTO DE KIMBERLEY']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'BAIRRO MULTIRÃO I']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'BONOPÓLIS']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'LONGITUDE HISTÓRICO']
# df_ocorrencias[df_ocorrencias['Municipio'] == 'MACAÚBA']


# Encontrar municipios
#df_ocorrencias[df_ocorrencias['Municipio'] == "NÃO IDENTIFICADA"]



In [18]:
# Excluindo valores
df_ocorrencias = df_ocorrencias[~df_ocorrencias['Municipio'].isin([
    'AGUASCALIENTES, MÉXICO',
    'ARGENTINA',
    'CANCUN, MÉXICO',
    'CANCÚN, MÉXICO',
    'CAPE TOWN',
    'CHITOSE, JAPÃO',
    'GUANAJUATO',
    'CIDADE DO MÉXICO, MÉXICO',
    'LISBOA',
    'MAR DEL PLATA',
    'MISSIONES',
    'ESTADOS UNIDOS',
    'FORA DO BRASIL',
    'GUANAJUATO, MÉXICO',
    'KUALA LUMPUR, MALÁSIA',
    'LISBOA/POTUGAL',
    'LISBOA ,PORTUGAL',
    'MADRID',
    'MAR DEL PLATA, ARGENTINA',
    'MIAMI',
    'MISSIONES,ARG',
    'MONTEVIDEO',
    'MORELOS, MÉXICO',
    'PARIS',
    'PEDRO JUAN CABALLERO',
    'SENEGAL, ÁFRICA',
    'SILAO, GUANAJUATO, MÉXICO',
    'SOMÁLIA, ÁFRICA'
])]

df_ocorrencias = df_ocorrencias.reset_index(drop=True) # Resetar o índice


df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202437239132', 'Municipio'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202346987620', 'Municipio'] = 'Petrópolis'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202389664945', 'Municipio'] = 'Santa Isabel'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '201107163514591', 'Municipio'] = 'Tabatinga'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '201705041722198/2017', 'Municipio'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == 'PT-VEB 11MAI2017 17H00 SERIPA3', 'Municipio'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == 'PU-PPY 28ABR2019 15H30 SERIPA1', 'Municipio'] = 'Peixoto de Azevedo'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202312268204', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202312268204', 'UF'] = 'SP'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '202312268204', 'Regiao'] = 'Sudeste'
df_ocorrencias.loc[df_ocorrencias['Numero_da_Ficha'] == '2022445310', 'Municipio'] = 'Rondolândia'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'AEROPORTO DE GUARULHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'AGUA BOA', 'Municipio'] = 'Água boa'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ÁGUAS CLARAS', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ALIANÇA DOS TOCANTINS', 'Municipio'] = 'Aliança do Tocantins'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ANAPÓLIS', 'Municipio'] = 'Anápolis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ARACAJÚ', 'Municipio'] = 'Aracaju'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ARIPUAÑA', 'Municipio'] = 'Aripuanã'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ASSIS CHATEAUBRIAN', 'Municipio'] = 'Assis Chateaubriand'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BALNEARIO CAMBORIU', 'Municipio'] = 'Balneário Camboriú'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BARRA DO GARCAS', 'Municipio'] = 'Barra do Garças'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BASE AÉREA DE SANTOS', 'Municipio'] = 'Santos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BELA VISTA DO PARAISO', 'Municipio'] = 'Bela Vista do Paraíso'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BELO HORIZONE', 'Municipio'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BIRITIBA MIRIM', 'Municipio'] = 'Biritiba-Mirim'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BLUMENAL', 'Municipio'] = 'Blumenau'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BOA VISTA DOS RAMOS', 'Municipio'] = 'Boa Vista do Ramos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BOM JESUS DE GOIÁS', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BONOPÓLIS', 'Municipio'] = 'Bonópolis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BRAISLÂNDIA', 'Municipio'] = 'Brasilãndia'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BRASILIA', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BRASÌLIA', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BRASÍLIA,', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'BRASÍLIADF', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAIXIAS DO SUL', 'Municipio'] = 'Caxias do Sul'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMOINAS', 'Municipio'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPINA GRANDE |', 'Municipio'] = 'Campina Grande'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPO DE MARTE'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPO DOS GOYTACAZES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPO GRANDEMS', 'Municipio'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPO GRNADE', 'Municipio'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPO MORÃO', 'Municipio'] = 'Campo Mourão'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPPINAS', 'Municipio'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPOS', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPOS DOS GOITACAZES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPOS DOS GOYTACAS', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAMPOS DOS GOYTACASES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CAPÃO DO CANOA', 'Municipio'] = 'Capão da Canoa'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CARLÂNDIA', 'Municipio'] = 'Indiara'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CASIMIRO E ABREU', 'Municipio'] = 'Casimirio de Abreu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CHAPADA DOS GUIMA', 'Municipio'] = 'Chapada dos Guimarães'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CIDADE DE UNA', 'Municipio'] = 'Una'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'COLIDER', 'Municipio'] = 'Colíder'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CONSELHEIRO LAFAYETE', 'Municipio'] = 'Conselheiro Lafaiete'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CUARI', 'Municipio'] = 'Coari'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CUIABA', 'Municipio'] = 'Cuiabá'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CURITBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CURITUBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'CURUTIBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ENTRE-IJUÍS', 'Municipio'] = 'Entre Ijuis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ESPINGARDA', 'Municipio'] = 'Eliseu Martins'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FLOARIANÓPOLIS', 'Municipio'] = 'Florianópolis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FOZ DO IGUAÇ', 'Municipio'] = 'Foz do Iguaçu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GAURULHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GAUÍRA', 'Municipio'] = 'Guaíra'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GOIANIA', 'Municipio'] = 'Goiânia'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GUARILHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GUARULHOSS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA BOA VISTA, QUIRINÓPOLIS', 'Municipio'] = 'Quirinópolis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA FORTALEZA DE SANTA TEREZINHA', 'Municipio'] = 'Santa Terezinha'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA FLORESTA, BRASNORTE', 'Municipio'] = 'Brasnorte'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA INHUMAS DO CHAPADÃO', 'Municipio'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA INHUMAS DO CHAPADÃO, MG', 'Municipio'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA NOVA AURORA, CAROLINA', 'Municipio'] = 'Carolina'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA NOVA ESPERANÇA', 'Municipio'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA NOVA ESPERANÇA, BALSAS', 'Municipio'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA SANTA BÁRBARA', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA SANTA BÁRBARA, BOM JESUS DE GOIÁS', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA SANTO INACIO', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA SANTO INÁCIO – RS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA TRES FIGUEIRAS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA TRÊS FIGUEIRAS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA UBIRATÃ', 'Municipio'] = 'Primavera do Leste'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'FAZENDA VIGOLO', 'Municipio'] = 'Palmas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'GRAMADINHO', 'Municipio'] = 'Seara'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'HANGAR LÍDER AVIAÇÃO', 'Municipio'] = 'Belém'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ILHÈUS'] = 'Ilhéus'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'IGARASSÚ'] = 'Igarassu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ITANHAEM'] = 'Itanhaém'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ITAPORANGA D’ AJUDA'] = "Itaporanga D'Ajuda"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'JACAREPAGUÁ'] = "Rio de Janeiro"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'JERICOACOARA'] = "Jijoca de Jericoacoara"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'JI-PARANÃ'] = "Ji-Paraná"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'JUINA'] = "Juína"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'JUNDIAI'] = "Jundiaí"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LENÇOIS'] = "Lençóis"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LENÇOIS PAULISTAS'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LENÇOIS PAULISTA'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LENÇÓIS PAULISTAS'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LONDRINA, PR'] = "Londrina"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LUÍS EDUARDO MAGAL', 'Municipio'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'LUIS EDUARDO MAGALHÃES', 'Municipio'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MANUEL URBANO', 'Municipio'] = 'Manoel Urbano'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MARICA', 'Municipio'] = 'Maricá'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MARILIA', 'Municipio'] = 'Marília'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MAÇAMBARÁ', 'Municipio'] = 'Maçambara'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MIGRANTINÓPOLIS', 'Municipio'] = 'Novo Horizonte do Oeste'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MIRASSOL D´OESTE', 'Municipio'] = "Mirassol D'Oeste"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MOGI GUAÇU', 'Municipio'] = "Mogi-Guaçu"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MOGI-MIRIM', 'Municipio'] = "Mogi Mirim"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'MONTES CLAROS,', 'Municipio'] = "Montes Claros"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NAVIRAI', 'Municipio'] = "Naviraí"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NITEROI', 'Municipio'] = "Niterói"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NOVA ARIPUANÃ', 'Municipio'] = 'Novo Aripuanã'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NOVA IGUAÇU', 'Municipio'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NOVA IGUAÇÚ', 'Municipio'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'NOVA MARINGA', 'Municipio'] = 'Nova Maringá'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PALIMIU', 'Municipio'] = 'Barcelos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PARATININGA', 'Municipio'] = 'Paranatinga'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PARATY', 'Municipio'] = 'Parati'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PARAUABEBAS', 'Municipio'] = 'Parauapebas'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PEDRA PRETA,', 'Municipio'] = 'Pedra Preta'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PONTES DE LACERDA', 'Municipio'] = 'Pontes e Lacerda'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PORO VELHO', 'Municipio'] = 'Porto Velho'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PORTO ESPERIDÃO', 'Municipio'] = 'Porto Esperidião '
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PRAIA DE ENSEADA - SÃO FRANCISCO DO SUL', 'Municipio'] = 'São Francisco do Sul'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'QUERENCIA DO NORTE', 'Municipio'] = 'Querência do Norte'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PORTO ALEGRE', 'Municipio'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PORTO ALEGRE |', 'Municipio'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'PIRAI DO SUL', 'Municipio'] = 'Piraí do Sul'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RECIFE', 'Municipio'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RECUFE', 'Municipio'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RIO DE JANEIRO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RIACHÂO DAS NEVES', 'Municipio'] = 'Riachão das Neves'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RIO DE JANERIO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RIO DE JANEIRO ', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RIO DE  JANEIRO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'RORAINOPOLIS', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == "SANT'ANA DO LIVRAMENTO", 'Municipio'] = 'Santana do Livramento'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == "SANTA CRUZ DAS PALMAS", 'Municipio'] = 'Santa Cruz das Palmeiras'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTANA DO PARAISO', 'Municipio'] = 'Santana do Paraíso'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTANA DO PARNAÍBA', 'Municipio'] = 'Santana de Parnaíba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTAREM', 'Municipio'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTA TEREZINHA DE JESUS', 'Municipio'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTA TEREZINHA DO ITAIPU', 'Municipio'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTA VITÓRIA DO PAMAR', 'Municipio'] = 'Santa Vitória Do Palmar'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTO ANTONIO DO LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTO ANTÔNIO DO LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSE DOS CAMPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSE DO RIO PRETO', 'Municipio'] = 'São José do Rio Preto'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO LUIZ', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO LUIZ', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTAÉM', 'Municipio'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SLAVADOR', 'Municipio'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃ PAULO', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO FELIX DO ARAGUAIA', 'Municipio'] = 'São Félix do Araguaia'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO FELIX DO XINGU', 'Municipio'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SAO PAULO', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOÃO DEL REY', 'Municipio'] = 'São João del Rei'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSE DOS PINHARES', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSÉ DOS PINHARES', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSÉ DO PINHAL', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO JOSÉ DOS CAMOPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SÃO LUIS', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SERRRANÓPOLIS', 'Municipio'] = 'Serranópolis'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == "SÍTIO SÃO JOSÉ, ZONA RURAL DE GLÓRIA D'OESTE", 'Municipio'] = "Glória D'Oeste"
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'TANGARÁ DAQ SERRA', 'Municipio'] = 'Tangaré da Serra'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'TELEMACO BORBA', 'Municipio'] = 'Telêmaco Borba'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'TIÊTE', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'TRAJANO DE MORAIS', 'Municipio'] = 'Trajano de Moraes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'TUPANCI', 'Municipio'] = 'São Sepé'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'URBERLÂNDIA', 'Municipio'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'VARZEA GRANDE', 'Municipio'] = 'Várza Grande'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'VERISSÍMO', 'Municipio'] = 'Veríssimo'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'VILA BELA DA SANTÍSSIMA', 'Municipio'] = 'Vila Bela da Santíssima Trindade'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'VITORIA DA CONQUISTA', 'Municipio'] = 'Vitória da Conquista'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'XINGUARÁ', 'Municipio'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'YARA', 'Municipio'] = 'Bandeirantes'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'XINGUARÁ', 'Municipio'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ÉRICO - AMAJARI', 'Municipio'] = 'Amajari'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'ÁREA RURAL', 'Municipio'] = 'Palestina'
df_ocorrencias.loc[df_ocorrencias['Municipio'] == 'SANTO ANTÔNIO DE LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'

In [14]:
df_ocorrencias[df_ocorrencias['Municipio'].isnull()]


,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
14,1359,562/2010,GOL,Incidente,2010-11-02,NaN,NaN,Indeterminado,NaN,NaN,NaN,NaN,NaN,NaN,COLISÃO COM PÁSSARO,PRGII,TPR,GOL,NaN,NaN,Voo Regular,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,737-7L9,L2J,B737,67996.0,148.0,BOEING COMPANY,verdadeiro
15,1294,485/2010,GOL,Ocorrência Anormal,2010-10-11,NaN,NaN,PR,Sul,NaN,NaN,NaN,NaN,NaN,COLISÃO COM PÁSSARO,PRGTG,TPR,GOL,NaN,NaN,Voo Regular,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,737-8EH,L2J,B738,79000.0,196.0,BOEING COMPANY,verdadeiro
16,975,353/2010,GOL,Ocorrência Anormal,2010-06-21,NaN,NaN,PR,Sul,NaN,NaN,NaN,NaN,NaN,COLISÃO COM PÁSSARO,PRGGG,TPR,GOL,NaN,NaN,Voo Regular,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,737-8EH,L2J,B738,74995.0,196.0,BOEING COMPANY,verdadeiro
21,5552,/2009,VIA AER,Acidente,2009-07-07,NaN,NaN,PR,Sul,FALHA OU MAU FUNCIONAMENTO DO MOTOR,NaN,NaN,NaN,NaN,NaN,N8540A,ESTRANGEIRA,VIA AER,SCF-PP,NaN,Voo Privado,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,falso
22,5376,/2001,NaN,Acidente,2001-06-05,NaN,NaN,Indeterminado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTJKL,TPP,NaN,NaN,NaN,Voo Privado,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,182P,L1P,C182,1338.0,4.0,CESSNA AIRCRAFT,verdadeiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11944,31577,2022746629,LIDER TÁXI AÉREO S.A. - AIR BRASIL,Acidente,2022-03-16,10:25,NaN,BA,Nordeste,PERDA DE CONTROLE EM VOO,Fora de Aeródromo,"-13,488","-38,8014",-,"A aeronave decolou do Aeroporto Internacional Deputado Luís Eduardo Magalhães (SBSV), Salvador, BA, por volta das 10h00min (UTC), com destino à plataforma Petrobras 69 (9PMN), Baía de Camamu, BA, a fim de realizar voo de táxi aéreo, com dois pilotos e onze passageiros a bordo. Durante a reta final, ocorreu a perda de controle em voo e consequente impacto da aeronave contra o mar.",PRLCT,TPX,LIDER TÁXI AÉREO S.A. - AIR BRASIL,LOC-I,Reta Final,Táxi Aéreo,Substancial,NaN,SBSV,1.0,NaN,NaN,NaN,NaN,NaN,1.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,S-76C,H2T,S76,5307.0,14.0,SIKORSKY AIRCRAFT,verdadeiro
12671,27363,202133489,VALDEMIR PEREIRA DE OLIVEIRA,Acidente,2021-02-26,11:00,NaN,Indeterminado,NaN,INDETERMINADO,Fora de Aeródromo,NaN,NaN,-,"A aeronave decolou de local não registrado, conhecido como 'Vila Sumaúma', no município de Mucajaí, RR, com destino ao local não registrado, conhecido como 'Prainha', no município de Alto Alegre, RR, por volta das 11h00min (UTC), com finalidade desconhecida, tendo um piloto a bordo. A aeronave não chegou ao destino e encontra-se desaparecida. Não existem informações sobre danos à aeronave. Não existem informações sobre lesões ao ocupante da aeronave.",PTDVB,TPP,VALDEMIR PEREIRA DE OLIVEIRA,UNK,Indeterminada,Voo Privado,Desconhecida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,182N,L1P,C182,1338.0,4.0,CESSNA AIRCRAFT,falso
12729,39036,202372705138,SWISS INTERNATIONAL AIR LINES,Incidente,2023-09-23,22:35:00,NaN,NaN,NaN,COLISÃO COM AVE,Fora de Aeródromo,"-23,2608","-46,2823",-,"A aeronave decolou do aero

In [15]:
df_municipios['MUNICÍPIO - TOM'] = df_municipios['MUNICÍPIO - TOM'].astype(str).str.strip().str.upper()
df_municipios[df_municipios['MUNICÍPIO - TOM'] == "LENÇÓIS PAULISTA"]

# df_municipios[df_municipios['MUNICÍPIO - TOM'].str.contains("LENÇÓIS PAULISTAS", na=False)]


,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
3885,6637,3526803,LENÇÓIS PAULISTA,Lençóis Paulista,SP


In [16]:
df_ocorrencias.head()

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,25043,202019110,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,Incidente,2020-06-17,22:10,ALTO ALEGRE,RR,Norte,OUTROS,SJYG,"2,765","-62,2214",Privado,"A ORIGEM E DESTINO ERAM DESCONHECIDOS, BEM COMO A FINALIDADE DO VOO. DE ACORDO COM INFORMAÇÕES DO OPERADOR, POR VOLTA DAS 18H10MIN LOCAL O PILOTO TEVE PANE NO GPS DA AERONAVE E SIMULTANEAMENTE ENCONTROU CONDIÇÕES METEOROLÓGICAS ADVERSAS NA ROTA, REALIZANDO UM POUSO DE PRECAUÇÃO NO AERÓDROMO ALTO MUCAJAÍ ( SJYG ), ALTO ALEGRE, RR, ÁREA INDÍGENA YANOMAMI. APÓS O POUSO A AERONAVE FOI DANIFICADA E O PILOTO DETIDO PELOS INDÍGENAS. A AERONAVE NÃO TEVE DANOS DURANTE A REALIZAÇÃO DO POUSO DE PRECAUÇÃO. O TRIPULANTE SAIU ILESO.",PTCZC,TPP,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,OTHR,Indeterminada,Voo Privado,Nenhum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,182K,L1P,C182,1270.0,4.0,CESSNA AIRCRAFT,verdadeiro
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,"-15,8692","-47,9208",Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
2,25503,202023366,AZUL,Incidente,2020-10-02,19:02,CAMPINAS,SP,Sudeste,FOGO/FUMAÇA,SBKP,"-23,0069","-47,1344",Público,"A AERONAVE DECOLOU DO AERÓDROMO VIRACOPOS (SBKP), CAMPINAS, SP, COM DESTINO AO AERÓDROMO HERCÍLIO LUZ (SBFL), FLORIANÓPOLIS, SC, A FIM DE TRANSPORTAR PESSOAL, COM CINCO TRIPULANTES E CENTO E TRINTA E NOVE PASSAGEIROS A BORDO.DURANTE A SUBIDA, OCORREU A INDICAÇÃO DE 'DUE MID EBAY SMOKE'. A TRIPULAÇÃO EFETUOU OS PROCEDIMENTOS NO QRH E DECIDIU POR RETORNAR PARA O AERÓDROMO DE ORIGEM.FOI DECLARADA EMERGÊNCIA. O POUSO FOI REALIZADO NORMALMENTE E SEM INTERCORRÊNCIAS.A AERONAVE NÃO TEVE DANOS.OS CINCO TRIPULANTES E OS CENTO E TRINTA E NOVE PASSAGEIROS SAÍRAM ILESOS.",PSAEA,TPR,AZUL LINHAS AEREAS BRASILEIRAS S.A,F-NI,Subida,Voo Regular,Nenhum,SBFL,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,139.0,NaN,NaN,NaN,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,verdadeiro
3,25623,202021754,JOAO MAURICIO GOTTARDI LOPES,Incidente,2020-10-07,15:30,ARAÇATUBA,SP,Sudeste,CONTATO ANORMAL COM A PISTA,SBAU,"-21,1442","-50,4264",Público,"A AERONAVE DECOLOU DO AERÓDROMO GOVERNADOR JOSÉ RICHA (SBLO), LONDRINA, PR, COM DESTINO AO AERÓDROMO DE ARAÇATUBA (SBAU), ARAÇATUBA, SP, PARA UM VOO PRIVADO, COM UM TRIPULANTE E UM PASSAGEIRO A BORDO. NA FINAL PARA O POUSO, SEGUNDO RELATO DO PILOTO, HOUVE UMA RAJADA DE VENTO E AO TOCAR A PISTA HOUVE O DESPRENDIMENTO DA RODA DA BEQUILHA. A AERONAVE TEVE DANOS LEVES. O PILOTO E O PASSAGEIRO SAÍRAM ILESOS.",PTJBJ,TPP,JOAO MAURICIO GOTTARDI LOPES,ARC,Pouso,Voo Privado,Leve,SBAU,SBLO,NaN,NaN,NaN,NaN,N